In [1]:
import matplotlib.pyplot as plt

from graded_asignments.graded_2.MLPEnsemble import MLPEnsemble
from graded_asignments.graded_2.DTForest import DTForest
from graded_asignments.graded_2.utils import *
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

#### 5a)
*Load the file “data_2022.csv” and create a test set in the same format as the training set you created in (2.a)*

In [2]:
pd_2021 = pd.read_csv('data/data_2021.csv')
pd_2022 = pd.read_csv('data/data_2022.csv')
np_2021 = pd_2021['Demand'].to_numpy()
np_2022 = pd_2022['Demand'].to_numpy()

#### 5b)
*Pick a model from (2, 3, or 4), and use this year’s demand so far to predict the missing values from 26.08.2022 to 31.12.2022. Save the predictions in a list (5 pts)*

In [3]:
forest_size = 200
window_size = int(len(np_2021)/4)
x, y = slide(np_2021, window_size)
single_tree = DecisionTreeRegressor()
dt_forest = DTForest(forest_size)
mlp_forest = MLPEnsemble(forest_size)
print('window size:',window_size,'\nforest size:',forest_size)

window size: 91 
forest size: 200


In [4]:
test_data, train_data = split([pd_2021, pd_2022])

In [5]:
single_tree.fit(x, y)  # training single tree

DecisionTreeRegressor()

In [6]:
dt_forest.train(train_data, window_size)

training...........................................................................................................................................................................................................Finished!

In [ ]:
mlp_forest.train(train_data, window_size)

training.....................

In [ ]:
# Initialize offset
offset = [None for i in range(len(np_2022))]
len(offset)

In [ ]:
# initial window is the last window of unseen data
initial_window = np_2022[-window_size:].reshape(1, -1).tolist()[0]
len(initial_window)

In [ ]:
data_2022 = np_2022.tolist()
n_predictions = 365 - len(np_2022)
dt_forest_2022_prediction = offset + dt_forest.predict_unseen(initial_window, n_predictions)
mlp_ensemble_2022_prediction = offset + mlp_forest.predict_unseen(initial_window, n_predictions)

In [ ]:
len(dt_forest_2022_prediction)
dt_check =len(dt_forest_2022_prediction) == 365
ml_check =len(mlp_ensemble_2022_prediction) == 365
print('Prediction length checks out:',(dt_check and ml_check))

#### 5c)
*Use matplotlib to plot the predictions for the remainder of 2022 (5 pts)*

In [ ]:
plt.plot(np_2022, label='Known data 2022')
plt.plot(dt_forest_2022_prediction, label='Decision Tree Forest Pred')
plt.plot(mlp_ensemble_2022_prediction, label='MLP Ensemble prediction')
plt.legend()